In [1]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [2]:
import numpy as np
from base.MachiKoro.env import *
#0 Array bias trong mỗi trận
#1 Array bias sau 100k trận 
#2 Lưu số trận 
#3 lưu hệ số đã dùng 
#4 Tỷ lệ thắng khi đấu 10k trận 
#5 Trigger để train 100k trận 
#6 Đếm số lần yếu liên tiếp
#7 Lưu array cho policy yếu 
#8 State của trận đấu
#9 Action của trận đấu
#10 Reward của trận đấu
#11 Số trận lưu state
#12 Trigger policy yếu

perx = [np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,getActionSize())),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,2)),np.zeros((1,2)),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((2000000,getStateSize())),np.zeros((2000000,1)),np.zeros((2000000,1)),np.zeros((1,1)),np.zeros((1,2))]
                         #0                                                      #1                   #2              #3                                                            #4              #5               #6                #7                                                               #8                                   #9               #10                       #11           #12

# Các hàm check tính chất của state


In [6]:
import numpy as np
@njit
def check_min(state, reward):
    just = np.zeros((2000000,2))
    for i in range(len(reward)):
        if reward[i] == -1:
            for j in range(i,len(reward)):
                if reward[j] != -1:
                    reward[i] = reward[j]
                    break
    # replace -1 with 0
    for i in range(len(reward)):
        if reward[i] == -1:
            reward[i] = 0
    # np.mean each array in the state and the corresponding reward
    for i in range(len(reward)):
        just[i][0] = np.mean(state[i])
        just[i][1] = (reward[i][0])
    # count the number of times each value of the state appears and the reward of each value of the state
    count = np.zeros((2000000,3))
    for i in range(len(reward)):
        for j in range(len(count)):
            if just[i][0] == count[j][0]:
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
            if count[j][0] == 0:
                count[j][0] = just[i][0]
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
    # find the index value the lowest rate of reward has the highest number of times appear
    min = 0
    for i in range(len(count)):
        if count[i][1] > count[min][1]:
            min = i
    for i in range(len(count)):
        if count[i][1] == count[min][1] and count[i][2] < count[min][2]:
            min = i

    # min = 10000000
    # min_index = 0
    # for i in range(len(count)):
    #     if count[i][1] != 0:
    #         if count[i][2]/count[i][1] < min:
    #             min = count[i][2]/count[i][1]
    #             min_index = i

    # print('mean',count[min][0])
    # print('frequency',count[min][1])
    # print('value',count[min][2])

    return count[min][0]
@njit
def check_weaker(min,x):
    if x == min:
        return 1
    else:
        return 0


# Agent Hoàn Chỉnh

In [7]:
@njit()
def p_bias(state,per):
    # Chuyển Mode gồm 3 mode mode 0 - train bias , mode 1 - test bias , mode 2 - train policy yếu ,mode 3 - test policy yếu,mode 4 - lưu state
    if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      # Lưu lại kết quả thắng
      per[5][0][1] = 0 
      per[5][0][1] += (per[4][0][0])
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Reset số lần yếu liên tiếp
      per[6][0][0] = 0 
    elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[6][0][0] +=1 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 

    # Bắt đầu mode lưu state
    if per[6][0][0] == 3 and per[11][0][0] < 2000000 :
      # Chuyển sang lưu state 
      per[5][0][0] = 4
      per[2][0][0] = 0
      # Rest số trận thắng và số trận đấu 
      per[4][0][1] = 0 
      per[4][0][0] = 0
      
    # Hết mode lưu state chuyển sang train policy yếu
    if per[5][0][0] == 4 and per[11][0][0] == 2000000:
      per[5][0][0] = 2

    # Bất đầu mode train và test policy yếu hơn
    if per[11][0][0] == 2000000:
      if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
        # Lưu lại kết quả thắng
        per[5][0][1] = 0 
        per[5][0][1] += (per[4][0][0])
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))

      elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
      
    # reset per[1][0] sau 100k trận
    if int(per[2][0][0]) == 100000 and per[11][0][0] < 2000000:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 1
    # reset per[1][0] sau 10k trận
    if int(per[2][0][0]) == 100000 and per[11][0][0] == 2000000:
      per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 3
      
    if per[11][0][0] == 2000000 and per[12][0][0] == 0:
      per[12][0][0] += check_min(per[8], per[10])

    # Chế độ train bias
    if int(per[2][0][0]) < 100000 and per[5][0][0] == 0  :
      actions = getValidActions(state)
      actions *= per[0][0]
      action = np.argmax(actions)
      if getReward(state) == 1 :
          per[1] += per[0][0]
      if getReward(state) == 0:
          per[0][0] = np.random.rand(getActionSize())

    # Chế độ test bias
    elif int(per[2][0][0]) <10000  and per[5][0][0] == 1 : 
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0])    
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  

    # Chế độ train policy yếu
    elif int(per[2][0][0]) < 100000 and per[5][0][0] == 2 and per[11][0][0] == 2000000 :
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      Check_yeu = check_weaker(per[12][0][0],np.mean(state))
      if Check_yeu == 1:
        per[12][0][1] = Check_yeu
        actions *= per[0][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1 and per[12][0][1] == 1:
        per[1] += per[0][0]
        per[12][0][1] = 0 
      if getReward(state) == 0 and per[12][0][1] == 1:
        per[0][0] = np.random.rand(getActionSize())
        per[12][0][1] = 0 

    # chế độ test policy yếu
    elif int(per[2][0][0]) <10000 and  per[5][0][0] == 3 and per[11][0][0] == 2000000:
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      if check_weaker(per[12][0][0],np.mean(state)) == 1:
        actions *= per[7][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  
    

    # Báo chuyển sang chế độ lưu state và action
    if per[5][0][0] == 4 and per[11][0][0] < 2000000:
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0]) 


    # Lưu state và action
    if getReward(state) != -2 and per[11][0][0] < 2000000 and per[5][0][0] == 4:
        for i in range(getStateSize()):
          per[8][int(per[11][0][0])][i] += state[i]        
        per[9][int(per[11][0][0])][0] += int(action)
        per[10][int(per[11][0][0])][0] += int(getReward(state))
        per[11][0][0] += 1

    # đếm số ván đấu 
    if getReward(state) != -1 :
        per[2][0][0] += 1
    return action,per

win, bias = numba_main_2(p_bias, 1000000,perx,0)

In [8]:
perx

[array([[0.91541877, 0.78138527, 0.43458889, 0.22867475, 0.77168499,
         0.17348561, 0.93550812, 0.23619529, 0.86690024, 0.50584476,
         0.66683865, 0.51208583, 0.00761461, 0.78072585, 0.18575613,
         0.69607765, 0.80544708, 0.60328132, 0.69767041, 0.62759108,
         0.88191368, 0.18301173, 0.36182282, 0.5366993 , 0.37151877,
         0.56393943, 0.60053527, 0.87077586, 0.08865304, 0.36381121,
         0.21928622, 0.76114021, 0.25635513, 0.2914422 , 0.11673072,
         0.24835089, 0.28314204, 0.63115091, 0.26502498, 0.44659228,
         0.19529139, 0.76490409, 0.16270095, 0.99485219, 0.30660902,
         0.83694822, 0.14792503, 0.54926396, 0.21615998, 0.90177593,
         0.67108411, 0.91074692, 0.62337567, 0.23152685]]),
 array([[30574.74962365, 31413.95578822, 30075.76668251, 30462.35579777,
         31106.36543289, 31104.60873316, 31421.34634024, 30673.54872276,
         30536.55229722, 30538.40228469, 30160.37091191, 30619.84818739,
         30610.99255165, 30878.

In [10]:
check_min(perx[8],perx[10])

0.9537037037037037

# Test lại tỷ lệ thắng của level 1 game MachiKoro

**Khi chưa khắc phục policy yếu :24551** 


**Khi đã khắc phục policy yếu :27961**

In [14]:
@njit()
def p_bias_1(state,per):
    actions = getValidActions(state)
    # Check xem policy có yếu không 
    if check_weaker(per[12][0][0],np.mean(state)) == 1:
      actions *= per[7][0]
      action = np.argmax(actions)
    else:
      actions *= per[3][0]
      action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_1, 100000,perx,1)
print(win)

# Find the most important value of index in the state that when it appear the policy dont work well


In [ ]:
def find_most_importane_feature(perx):
    import pandas as pd

    df = pd.DataFrame(perx[8])
    df['action'] = perx[9]
    df['reward'] = perx[10]
    for i in range(len(df)):
        if df['reward'][i] == -1:
            for j in range(i,len(df)):
                if df['reward'][j] != -1:
                    df['reward'][i] = df['reward'][j]
                    break
    df = df[df['reward'] != -1]

    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier

    # Load the state-action dataset

    # Split the data into features (state properties) and target (outcome)
    features = df.drop("reward", axis=1)
    target = df["reward"]

    # Train a Random Forest Classifier on the data
    clf = RandomForestClassifier()
    clf.fit(features, target)

    # Get the feature importances
    importances = clf.feature_importances_

    # Find the property (x) of the state that is most important for the policy's performance
    x = features.columns[importances.argmax()]
    # Plot the top  feature importances 
    import matplotlib.pyplot as plt
    import numpy as np
    indices = np.argsort(importances)[::-1]
    plt.figure(figsize=(10, 5))
    plt.title("Feature importances")
    plt.bar(range(features.shape[1]), importances[indices], color="r", align="center")
    plt.xticks(range(features.shape[1]), features.columns[indices], rotation=90)
    plt.xlim([-1, features.shape[1]])
    plt.show()
    most_imp_feature = None
    lowest_percentage = 100
    print('feature importances',x)
    for i in df[x].unique():
        current_percentage = df[df[x] == i]['reward'].value_counts()[1]/df[df[x] == i]['reward'].value_counts().sum()
        if current_percentage < lowest_percentage:
            most_imp_feature = i
            lowest_percentage = current_percentage

    print("The value of the most important feature with the lowest percentage of value 1 in reward is:", most_imp_feature)#  all the below code in one function    
    return most_imp_feature


In [9]:
i = 0 +10
d = 0 
while 0<i:
    i = i-1
    d= d+1